In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import os
import itertools
import sklearn
import pandas as pd
import scipy.stats as stats

#from keras import Sequential
#from keras.layers import Dense
#from keras.wrappers.scikit_learn import KerasRegressor

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression



In [2]:
neuron_data = {}

files = sorted(os.listdir('../neuron_logs/train_data'))

In [3]:
def reduce_to_statistics(activations, labels, debug=False):
    sorted_data = []
    for i in range(10): #hardcoded MOFO
        sorted_data += [[]]
    for i, a in zip(labels, activations):
        sorted_data[i] += [a]
    if debug:
        return(sorted_data)
    statistics = []
    for ar in sorted_data:
        curr_stats = stats.describe(ar)
        statistics += [curr_stats.mean, curr_stats.variance, curr_stats.skewness, curr_stats.kurtosis, curr_stats.minmax[0],
                       curr_stats.minmax[1], curr_stats.nobs]
        #print(statistics)
    return(statistics)

In [4]:
features, labels = [], []
target = 'usefulness_loss'
fin = 10
activations_no = 2000
for i, filename in enumerate(files[1:]):
    with open(os.path.join('..', 'neuron_logs', 'train_data', filename), 'r') as f:
        print(filename)
        neuron_data = json.load(f)
    if i == 10:
        break
    for e in neuron_data.keys():
        if e == '0' or int(e) > fin:
            #print(e)
            continue
        for neuron in neuron_data[e]:
            if ' ' not in neuron:
                continue
            current_data = neuron_data[e][neuron]
            important_features = []
            important_features = current_data['activations'][:activations_no]
            important_features += reduce_to_statistics(current_data['activations'], neuron_data[e]['original_labels'])
            important_features += [current_data['depth']]
            important_features += [current_data['inverse_depth']]
            important_features += [current_data['width']]
            # important_features += [current_data['input_weights']]
            # important_features += [current_data['output_weights']]
            important_features += [current_data['reg_loss_in_layer']]
            important_features += [e]
            features += [important_features]
            #labels += [current_data[target]]
            labels += [current_data[target]]

output_20191015-203714.json
output_20191015-215448.json
output_20191015-231129.json
output_20191016-002746.json
output_20191016-014510.json
output_20191016-030245.json
output_20191016-042123.json
output_20191016-053821.json
output_20191016-065412.json
output_20191016-081229.json
output_20191016-093233.json


In [9]:
def extract_data(filename, fin = 10, activations_no = 1000, target = 'usefulness_loss', shuffle = True):
    
    features, labels = [], []
    with open(os.path.join('..', 'neuron_logs', 'train_data', filename), 'r') as f:
        neuron_data = json.load(f)
        for e in neuron_data.keys():
            if e == '0' or int(e) > fin:
                continue
            for neuron in neuron_data[e]:
                if ' ' not in neuron:
                    continue
                current_data = neuron_data[e][neuron]
                important_features = []
                important_features = current_data['activations'][:activations_no]
                important_features += reduce_to_statistics(current_data['activations'], neuron_data[e]['original_labels'])
                important_features += [current_data['depth']]
                important_features += [current_data['inverse_depth']]
                important_features += [current_data['width']]
                # important_features += [current_data['input_weights']]
                # important_features += [current_data['output_weights']]
                important_features += [current_data['reg_loss_in_layer']]
                important_features += [e]
                features += [important_features]
                #labels += [current_data[target]]
                labels += [current_data[target]]
    
    if shuffle:
        shuffled = sklearn.utils.shuffle(np.concatenate([np.array(features), np.array(labels).reshape(-1, 1)], axis=1))
        features, labels = shuffled[:, :-1], shuffled[:, -1]
    
    return(np.array(features, dtype=np.float32), np.array(labels, dtype=np.float32))


In [5]:
features = np.array(features)
labels = np.array(labels)
print(features.shape, labels.shape)

(12000, 2075) (12000,)


In [6]:
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [7]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=0)

X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

testscaler = StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)

In [8]:
print(f"{X_train.shape} {X_test.shape} {y_train.shape} {y_test.shape}")
reg = linear_model.LinearRegression(n_jobs = -1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
mean_squared_error(y_pred, y_test)

(9600, 2075) (2400, 2075) (9600,) (2400,)


0.014492879

In [11]:
X_test, y_test = extract_data(files[0], activations_no=2000)
scaler = StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)
y_pred = reg.predict(X_test)
print(f"MSE on unknown network: {mean_squared_error(y_pred, y_test)}")

MSE on unknown network: 0.18315640091896057


In [12]:
reg = linear_model.Ridge(alpha = 0.5)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
mean_squared_error(y_pred, y_test)

/home/levai/.miniconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/linear_model/ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=9.26344e-09): result may not be accurate.
  overwrite_a=True).T


0.0942055

In [13]:
y_pred = reg.predict(X_test)
print(f"MSE on unknown network: {mean_squared_error(y_pred, y_test)}")

MSE on unknown network: 0.09420550614595413


In [15]:
reg = linear_model.Lasso(alpha=0.1)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(mean_squared_error(y_pred, y_test))
print(f"MSE on unknown network: {mean_squared_error(y_pred, y_test)}")

0.04742748
MSE on unknown network: 0.047427479177713394


In [17]:
reg = linear_model.LassoCV(cv=5, random_state=0)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f"MSE on unknown network: {mean_squared_error(y_pred, y_test)}")

/home/levai/.miniconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.04706106334924698, tolerance: 0.035513896495103836
  tol, rng, random, positive)
/home/levai/.miniconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.045893024653196335, tolerance: 0.035513896495103836
  tol, rng, random, positive)
/home/levai/.miniconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06305776536464691, tolerance: 0.035513896495103836
  tol, rng, random, positive)
/home/levai/.miniconda3/envs/tensorflow/lib/python3.7/site-p

MSE on unknown network: 0.03522554039955139


/home/levai/.miniconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 32.51370620727539, tolerance: 0.04467388987541199
  positive)


In [18]:
reg = linear_model.RidgeCV(alphas=np.logspace(-6, 6, 13))
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
print(f"MSE on unknown network: {mean_squared_error(y_pred, y_test)}")

MSE on unknown network: 0.07719597931392547


In [19]:
from sklearn.datasets import make_classification
from torch import nn
import torch
import torch.nn.functional as F
import torch.optim as optim


import skorch
from skorch import NeuralNetClassifier
from skorch import NeuralNetRegressor

In [24]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=1000,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(X_train.shape[1], num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, num_units//2)
        self.dense2 = nn.Linear(num_units//2, num_units//2)
        self.dense3 = nn.Linear(num_units//2, num_units//2)
        self.denselast = nn.Linear(num_units//2, 100)
        self.output = nn.Linear(100, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = F.relu(self.dense2(X))
        X = F.relu(self.dense3(X))
        X = F.relu(self.denselast(X))
        X = self.output(X)
        return X

In [25]:
from skorch.callbacks import LRScheduler

lrscheduler = LRScheduler(monitor='train_loss',
            policy='ReduceLROnPlateau', # step_size=15, gamma=0.5)
            mode='min', factor=0.2, patience=10, verbose=True, 
            threshold=0.00001, threshold_mode='rel', 
            cooldown=0, min_lr=0, eps=1e-08)

from skorch.callbacks import Checkpoint

checkpoint = Checkpoint(
    f_params='best_model.pt', monitor='valid_loss_best')

from skorch.callbacks import EarlyStopping

earlystopping = EarlyStopping(
    monitor='train_loss',
    patience=20,
    threshold=0.0001,
    threshold_mode='abs',
    lower_is_better=True,)

In [26]:
net = NeuralNetRegressor(
    RegressorModule,
    max_epochs=200,
    lr=0.1,
    optimizer=optim.SGD,
    optimizer__momentum=0.9,
    # callbacks=[skorch.callbacks.LRScheduler(skorch.callbacks.WarmRestartLR)],
    callbacks = [lrscheduler, checkpoint, earlystopping],
    warm_start=False
)

In [27]:
net.fit(X_train, y_train.reshape(-1, 1))

  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.0987        0.0307     +  3.3803
      2        0.0281        0.0257     +  2.9467
      3        0.0237        0.0214     +  2.8305
      4        0.0218        0.0208     +  2.9731
      5        0.0198        0.0221        3.0816
      6        0.0186        0.0214        3.0854
      7        0.0173        0.0212        3.0655
      8        0.0163        0.0205     +  3.0977
      9        0.0148        0.0189     +  2.9241
     10        0.0136        0.0172     +  2.9117
     11        0.0127        0.0177        2.9575
     12        0.0116        0.0173        3.6116
     13        0.0119        0.0193        3.5493
     14        0.0119        0.0213        3.3963
     15        0.0112        0.0164     +  3.0056
     16        0.0095        0.0112     +  3.2633
     17        0.0082        0.0097     +  5.1497
     18        0.0074        0.0073     +  4.3055


<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=RegressorModule(
    (dense0): Linear(in_features=2075, out_features=1000, bias=True)
    (dense1): Linear(in_features=1000, out_features=500, bias=True)
    (dense2): Linear(in_features=500, out_features=500, bias=True)
    (dense3): Linear(in_features=500, out_features=500, bias=True)
    (denselast): Linear(in_features=500, out_features=100, bias=True)
    (output): Linear(in_features=100, out_features=1, bias=True)
  ),
)

In [28]:
y_pred = net.predict(X_test)
print(f"MSE on unknown network: {mean_squared_error(y_pred, y_test)}")

MSE on unknown network: 0.03272349759936333


In [29]:
#valid_acc = [x['valid_acc'] for x in net.history]
train_loss = [x['train_loss'] for x in net.history]
valid_loss = [x['valid_loss'] for x in net.history]
fig, ax1 = plt.subplots(figsize = (12, 8), facecolor = 'w')

# ax1.plot(range(len(valid_acc)), valid_acc, color='firebrick', label = 'neural net validation acc')


ax1.plot(range(len(valid_loss)), [reg.score(X_test, y_test)]*len(valid_loss),  '--', color = 'blue', label = 'logreg test set acc')
ax1.plot(range(len(valid_loss)), [net.score(X_test, y_test)]*len(valid_loss), '--', color = 'firebrick', label = 'neural net test set acc')


ax2 = ax1.twinx()
ax2.plot(range(len(valid_loss)), train_loss, '--', alpha = 0.5, color='firebrick', label = 'neural net train loss')
ax2.plot(range(len(valid_loss)), valid_loss, '--', color='red', label = 'neural net validation loss')
ax1.set_xlabel('epoch')
ax1.set_ylabel('accuracy')
ax2.set_ylabel('train loss')
ax1.grid(True)
ax1.legend(loc = 3)
ax2.legend(loc = 4)

KeyError: 'train_loss'